In [ ]:
import pandas as pd
import requests

from api.data import (store_database_for_eys_gene,
                      parse_lovd,
                      parse_gnomad,
                      LOVD_PATH,
                      set_lovd_dtypes,
                      set_gnomad_dtypes,
                      request_gnomad_api_data,
                      merge_gnomad_lovd,
                      GNOMAD_PATH,
                      )
from api.data import save_lovd_as_vcf


pd.options.display.max_columns = 0

In [ ]:
store_database_for_eys_gene("lovd", override=False)

In [ ]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")

In [ ]:
gnomad_data = request_gnomad_api_data("EYS")

display(gnomad_data)

In [ ]:
store_database_for_eys_gene('gnomad', False)

gnomad_data_2 = parse_gnomad(GNOMAD_PATH +'/gnomad_data.csv')

In [ ]:
display(gnomad_data_2)

In [ ]:
gnomad_data_2.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_downloaded.csv', index=False)
gnomad_data.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_api.csv', index=False)

In [ ]:
len(gnomad_data_2), len(gnomad_data)

print(len(gnomad_data_2) - len(gnomad_data))

In [ ]:
gnomad_data

In [ ]:
missing_from_api = []

for i in gnomad_data['gnomAD ID']:
    if(i in gnomad_data_2['gnomAD ID'].values):
        continue
    missing_from_api.append(i)

len(missing_from_api)

missing_data = gnomad_data.loc[gnomad_data['gnomAD ID'].isin(missing_from_api)]

missing_data

In [ ]:
missing_data.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_missing.csv', index=False)

In [ ]:
set_lovd_dtypes(data)
set_gnomad_dtypes(gnomad_data)

variants_on_genome = data["Variants_On_Genome"].copy()

lovd_data = pd.merge(data["Variants_On_Transcripts"],
                       variants_on_genome[['id','VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
final_data = merge_gnomad_lovd(lovd_data, gnomad_data)
final_data

In [ ]:
for i in data:
    print(i)
    display(data[i])

In [ ]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())

In [ ]:
save_lovd_as_vcf(data["Variants_On_Genome"], "./lovd.vcf")

In [ ]:
from subprocess import Popen

process = Popen("spliceai -I ./lovd.vcf -O ./lovd_output.vcf -R ../tools/spliceai/hg38.fa -A grch38".split())
process.wait()

In [ ]:
from api.tools import get_revel_scores

chromosome = 6
position = 65655758

results = get_revel_scores(chromosome, position)

display(results)

In [1]:
from api.data.refactoring import merge_gnomad_lovd, parse_gnomad, set_gnomad_dtypes
import pandas as pd
from api import (store_database_for_eys_gene,
                 parse_lovd,
                 set_lovd_dtypes,
                 LOVD_PATH,
                 GNOMAD_PATH)

store_database_for_eys_gene('lovd', False)
store_database_for_eys_gene('gnomad', False)

lovd_data = parse_lovd(LOVD_PATH + "/lovd_data.txt")
gnomad_data = parse_gnomad(GNOMAD_PATH+'/gnomad_data.csv')

set_lovd_dtypes(lovd_data)
set_gnomad_dtypes(gnomad_data)

variants_on_genome = lovd_data["Variants_On_Genome"].copy()

lovd_data = pd.merge(lovd_data["Variants_On_Transcripts"],
                       variants_on_genome[['id','VariantOnGenome/DNA','VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
final_data = merge_gnomad_lovd(lovd_data, gnomad_data)
final_data

The file at ../data/lovd/lovd_data.txt already exists.


,id,transcriptid,effectid,position_c_start,position_c_start_intron,position_c_end,position_c_end_intron,VariantOnTranscript/DNA,VariantOnTranscript/RNA,VariantOnTranscript/Protein,...,Homozygote Count Amish_gnomad,Hemizygote Count Amish_gnomad,Allele Count South Asian_gnomad,Allele Number South Asian_gnomad,Homozygote Count South Asian_gnomad,Hemizygote Count South Asian_gnomad,Allele Count Remaining_gnomad,Allele Number Remaining_gnomad,Homozygote Count Remaining_gnomad,Hemizygote Count Remaining_gnomad
0,822823,7329,70,632,0,632,0,c.632G>A,r.(?),p.(Cys211Tyr),...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,822787,7329,70,8391,0,8391,0,c.8391del,r.(?),p.(Gly2799Valfs*31),...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,822843,7329,70,5608,0,5608,0,c.5608C>T,r.(?),p.(Arg1870Trp),...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,822771,7329,70,8206,0,8206,0,c.8206G>C,r.(?),p.(Ala2736Pro),...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0,0,0,55362,0,0,0,44082,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13218,959060,7329,70,9383,0,9387,0,c.9383_9387del,r.(?),p.(Lys3128ArgfsTer7),...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
13219,959064,7329,50,0,0,0,0,c.-538_862+10652{1}inv,r.?,p.?,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
13220,985494,7329,70,2137,20590,3444,-29847,c.2137+20590_3444-29847del,r.?,p.(Val713AspfsTer14),...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
13221,986425,7329,90,4361,0,4362,0,c.4361_4362delinsAG,r.(?),p.(Ser1454Ter),...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
